# **GC4 Phase 1**

Name: Julio Putra David

Batch: 10

______________

## **1. Introduction**

This notebook will cover the establishment of unsupervised machine learning model for a customer segmentation to define marketing strategy. The Dataset contains the usage behavior of about 9000 active credit card holders during the last 6 months.

#### **Attribute Information**

* `CUSTID` : Identification of Credit Card holder (Categorical)
* `BALANCE` : Balance amount left in their account to make purchases
* `BALANCEFREQUENCY` : How frequently the Balance is updated, score between 0 and 1 (1 = frequently updated, 0 = not frequently updated)
* `PURCHASES` : Amount of purchases made from account
* `ONEOFFPURCHASES` : Maximum purchase amount done in one-go
* `INSTALLMENTSPURCHASES` : Amount of purchase done in installment
* `CASHADVANCE` : Cash in advance given by the user
* `PURCHASESFREQUENCY` : How frequently the Purchases are being made, score between 0 and 1 (1 = frequently purchased, 0 = not frequently purchased)
* `ONEOFFPURCHASESFREQUENCY` : How frequently Purchases are happening in one-go (1 = frequently purchased, 0 = not frequently purchased)
* `PURCHASESINSTALLMENTSFREQUENCY` : How frequently purchases in installments are being done (1 = frequently done, 0 = not frequently done)
* `CASHADVANCEFREQUENCY` : How frequently the cash in advance being paid
* `CASHADVANCETRX` : Number of Transactions made with "Cash in Advanced"
* `PURCHASESTRX` : Numbe of purchase transactions made
* `CREDITLIMIT` : Limit of Credit Card for user
* `PAYMENTS` : Amount of Payment done by user
* `MINIMUM_PAYMENTS` : Minimum amount of payments made by user
* `PRCFULLPAYMENT` : Percent of full payment paid by user
* `TENURE` : Tenure of credit card service for user

## **2. Import Libraries**

In [ ]:
# Basic Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 

# Pre-processing Libraries
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from feature_engine.outliers import Winsorizer, OutlierTrimmer
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score,silhouette_samples
import matplotlib.cm as cm
import scipy.cluster.hierarchy as shc

# Clustering Libraries
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

# Warning Neglect Library
import warnings
warnings.filterwarnings('ignore')

## **3. Data Loading**

In [ ]:
df = pd.read_csv('CC_GENERAL.csv')
df.head()

In [ ]:
r = df.shape[0]
c = df.shape[1]
print('Number of rows    =', r)
print('Number of columns =', c)

In [ ]:
df.info()

Make a copy of the original dataset

In [ ]:
data = df.copy()

## **4. Exploratory Data Analysis (EDA)**

#### **1. List of numerical columns**

In [ ]:
num_data = data.select_dtypes(include=np.number).columns.tolist()
num_data

#### **2. List of Categorical Columns**

In [ ]:
data.select_dtypes(include=['object']).columns.tolist()

`CUST_ID` will be dropped away because it does not show any pattern for clustering.

#### **3. Descriptive Statistics**

In [ ]:
data.describe()

There is no oddity in the descriptive statistics of each column.

#### **4. Missing Values Detection**

In [ ]:
data.isnull().any()

We have several missing values in `CREDIT_LIMIT`, `MINIMUM_PAYMENTS`, and `TENURE`.

#### **5. Distribution of Features**

In [ ]:
# This is the function to check the distribution of the columns

def skew_check(data, column):
    skewness = data[column].skew(axis = 0, skipna = True)
    if skewness <= 0.5 and skewness >= -0.5:
        print(f'[Gaussian] Skewness of {column} =', skewness)
    else:
        print(f'[Skewed] Skewness of {column} =', skewness)

In [ ]:
for feature in num_data:
    skew_check(data, feature)

In [ ]:
plt.figure(figsize=(14,8))
for i in range(0,len(num_data)):
    plt.subplot(4,5, i+1)
    sns.histplot(x=data[num_data[i]], color='green')
    plt.tight_layout()

Almost all columns are skewed. Only `PURCHASE_FREQUENCY` that has normal distribution.

From the histogram, it seems that `TENURE` is a categorical column.

#### **6. Boxplot Analysis**

In [ ]:
plt.figure(figsize=(14,8))
for i in range(0,len(num_data)):
    plt.subplot(4,5, i+1)
    sns.boxplot(x=data[num_data[i]], color='green')
    plt.tight_layout()

We have many outliers in several columns. It will be handled later on in the Data Pre-processing section.

#### **7. Tenure**

In [ ]:
data.TENURE.unique()

`Tenure` is a categorical data.

#### **8. Pearson Correlation Analysis**

In unsupervised learning, we will check the multicollinearity of each feature simultaneously using Variance Inflation Factor (VIF). Now we want to see first the correlation between features using the Pearson Correlation method.

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(data.corr(), annot=True, cmap='Blues')
plt.title('Pearson Correlation Analysis')
plt.show()

From the heatmap above it appears that some features have strong and also weak correlation and with each other. Strong correlation means the feature correlate well with other feature. On the other hand, in unsupervised learning we don't want the features to have strong correlation with other features because we don't want to input features that have similar characteristic with each other so that our model would be lighter and simpler.

## **5. Data Pre-processing**

### **5.1 Dropping Features**

As mentioned in EDA, we will drop `CUST_ID`

In [ ]:
data.drop('CUST_ID', axis=1, inplace=True)

### **5.2 Inference Dataset**

We will take 10 samples out to be our data inference. 

In [ ]:
data_inf = data.sample(10, random_state=77)

In [ ]:
data_train = data.drop(data_inf.index)

To prevent unwanted error, we will reset the index.

In [ ]:
data_train.reset_index(drop=True, inplace=True)
data_inf.reset_index(drop=True, inplace=True)

### **5.3 Handling Outliers**

In [ ]:
# This is the function to detect how many outliers in each column

def detect_otl(data, column):
    skewness = data[column].skew(axis = 0, skipna = True)
    if skewness <= 0.5 and skewness >= -0.5:
        upper_boundary = data[column].mean() + 1.5 * data[column].std()
        lower_boundary = data[column].mean() - 1.5 * data[column].std()
        print(f'[Gaussian] Skewness of {column} =', skewness)
        print('% above upper boundary : {}'.format(len(data[data[column] > upper_boundary]) / len(data) * 100))
        print('% below lower boundary : {}'.format(len(data[data[column] < lower_boundary]) / len(data) * 100))
        print('-'*75)
    else:
        IQR = data[column].quantile(0.75) - data[column].quantile(0.25)
        lower_boundary = data[column].quantile(0.25) - (IQR * 1.5)
        upper_boundary = data[column].quantile(0.75) + (IQR * 1.5)
        print(f'[Skewed] Skewness of {column} =', skewness)
        print('% above upper boundary : {}'.format(len(data[data[column] > upper_boundary]) / len(data) * 100))
        print('% below lower boundary : {}'.format(len(data[data[column] < lower_boundary]) / len(data) * 100))
        print('-'*75)

In [ ]:
for i in num_data:
    detect_otl(data_train, i)

Below is the features whose outliers will be capped and trimmed.

In [ ]:
otl_cap = ['MINIMUM_PAYMENTS', 'PAYMENTS', 'PURCHASES_TRX', 'CASH_ADVANCE_TRX', 'CASH_ADVANCE_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
           'CASH_ADVANCE', 'INSTALLMENTS_PURCHASES', 'ONEOFF_PURCHASES', 'PURCHASES', 'BALANCE']
otl_trim = ['CREDIT_LIMIT']

In [ ]:
winsorizer = Winsorizer(capping_method='iqr', # We use IQR because all of the features are skewed
                        tail='both',
                        fold=1.5,
                        variables=otl_cap,
                        missing_values='ignore')

data_train_capped = winsorizer.fit_transform(data_train)

In [ ]:
outlier_trimmer = OutlierTrimmer(capping_method='iqr', # We use IQR because all of the features are skewed
                                 tail='both', 
                                 fold=1.5, 
                                 variables=otl_trim, 
                                 missing_values='ignore')

data_train_trimmed = outlier_trimmer.fit_transform(data_train_capped)
print('Size dataset - Before trimming : ', data_train_capped.shape)
print('Size dataset - After trimming  : ', data_train_trimmed.shape)

### **5.4 Handling Missing Values**

First, we will check in which column the null values lies in.

In [ ]:
data_train_trimmed.isnull().any()

We have null values in `CREDIT_LIMIT` and `MINIMUM_PAYMENTS`

We will do imputation on all occurrences of missing values with:
* **Mean** : if the variable has a **Normal/Gaussian distribution**.
* **Median** : if the variable has a **skewed distribution**.

In [ ]:
def impute_na_num(data, variable):
    skewness = data[variable].skew(axis = 0, skipna = True)
    if skewness <= 0.5 and skewness >= -0.5:
        data[variable].fillna(data[variable].mean(), inplace=True)
    else:
        data[variable].fillna(data[variable].median(), inplace=True)

    return data

In [ ]:
for i in ['CREDIT_LIMIT', 'MINIMUM_PAYMENTS']:
    data_train_ipt = impute_na_num(data_train_trimmed, i)

In [ ]:
data_train_ipt.isnull().any()

In [ ]:
data_train_ipt.reset_index(drop=True, inplace=True)

### **5.5 Multicollinearity**

Now we will check the correlation between features using VIF.

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = data_train_ipt.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(data_train_ipt.values, i) for i in range(len(data_train_ipt.columns))]

vif_data

VIF less than 5: `PRC_FULL_PAYMENT`, `CREDIT_LIMIT`, `PAYMENTS`

We are expecting to have 4 features after PCA.

### **5.6 Feature Scaling**

`TENURE` is a categorical data, thus we will exclude it from feature scaling.

In [ ]:
data_for_scaling = data_train_ipt.copy()
data_for_scaling.drop('TENURE', axis=1,inplace=True)

In [ ]:
standard_scaler = StandardScaler()

data_train_scaled = standard_scaler.fit_transform(data_for_scaling)

In [ ]:
data_train_scaled_df = pd.DataFrame(data_train_scaled, columns=data_for_scaling.columns)
data_train_scaled_df['TENURE'] = data_train_ipt['TENURE']

In [ ]:
data_train_final = data_train_scaled_df.to_numpy()
data_train_final.shape

### **5.7 PCA (Principal Component Analysis)** 

We will take 80% information of the whole dataset. So we will use `n_components = 0.8`

In [ ]:
pca_1 = PCA(n_components=0.8).fit(data_train_final)
pca_1.explained_variance_ratio_

##### **Explained Variance ratio**

In [ ]:
fig,ax=plt.subplots(ncols=2,figsize=(16,5))
ax[0].plot(range(1,7),pca_1.explained_variance_ratio_)
ax[0].set_xlabel('Component')
ax[0].set_ylabel('Explained Variance Ratio')

ax[1].plot(range(1,7),np.cumsum(pca_1.explained_variance_ratio_))
ax[1].set_xlabel('Number of Component')
ax[1].set_ylabel('Cummulative Explained Var Ratio')
plt.show()

The Cummulative EVR shows that 80% informations needs 6 features.

We will make a variable that contains 80% data.

In [ ]:
data_train_reduced = pca_1.transform(data_train_final)
data_train_reduced.shape

From the analysis above, it appears that 80% information needs 6 features.

## **6. Model Definition**

In [ ]:
def plot_silhouette(range_n_clusters,X):
    for n_clusters in range_n_clusters:
        # Create a subplot with 1 row and 2 columns
        fig, (ax1, ax2) = plt.subplots(1, 2)
        fig.set_size_inches(15, 4)

        # The 1st subplot is the silhouette plot
        # The silhouette coefficient can range from -1, 1 but in this example all
        # lie within [-0.1, 1]
        ax1.set_xlim([-0.1, 1])
        # The (n_clusters+1)*10 is for inserting blank space between silhouette
        # plots of individual clusters, to demarcate them clearly.
        ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

        # Initialize the clusterer with n_clusters value and a random generator
        # seed of 10 for reproducibility.
        clusterer = KMeans(n_clusters=n_clusters, random_state=10)
        cluster_labels = clusterer.fit_predict(X)

        # The silhouette_score gives the average value for all the samples.
        # This gives a perspective into the density and separation of the formed
        # clusters
        silhouette_avg = silhouette_score(X, cluster_labels)
        print("For n_clusters =", n_clusters,
              "The average silhouette_score is :", silhouette_avg)

        # Compute the silhouette scores for each sample
        sample_silhouette_values = silhouette_samples(X, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            # Aggregate the silhouette scores for samples belonging to
            # cluster i, and sort them
            ith_cluster_silhouette_values = \
                sample_silhouette_values[cluster_labels == i]

            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_clusters)
            ax1.fill_betweenx(np.arange(y_lower, y_upper),
                              0, ith_cluster_silhouette_values,
                              facecolor=color, edgecolor=color, alpha=0.7)

            # Label the silhouette plots with their cluster numbers at the middle
            ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

            # Compute the new y_lower for next plot
            y_lower = y_upper + 10  # 10 for the 0 samples

        ax1.set_title("The silhouette plot for the various clusters.")
        ax1.set_xlabel("The silhouette coefficient values")
        ax1.set_ylabel("Cluster label")

        # The vertical line for average silhouette score of all the values
        ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

        ax1.set_yticks([])  # Clear the yaxis labels / ticks
        ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

        # 2nd Plot showing the actual clusters formed
        colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
        ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                    c=colors, edgecolor='k')

        # Labeling the clusters
        centers = clusterer.cluster_centers_
        # Draw white circles at cluster centers
        ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                    c="white", alpha=1, s=200, edgecolor='k')

        for i, c in enumerate(centers):
            ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                        s=50, edgecolor='k')

        ax2.set_title("The visualization of the clustered data.")
        ax2.set_xlabel("Feature space for the 1st feature")
        ax2.set_ylabel("Feature space for the 2nd feature")

        plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                      "with n_clusters = %d" % n_clusters),
                     fontsize=14, fontweight='bold')

### **6.1 Kmeans Clustering**

#### **6.1.1 Finding The Best Number of Cluster for Kmeans**

##### **Elbow Method**

In [ ]:
K=[2,3,4,5,6,7,8]
inertia=[KMeans(n_clusters=i).fit(data_train_reduced).inertia_ for i in K]
plt.plot(K,inertia, marker='o', color='r')
plt.xlabel('K')
plt.ylabel('Inertia')

It's hard to determine where the elbow is when the plot is bending smoothly like the plot above. Therefore, we will check the silhouette score and plot.

##### **Siluet Score & Plot**

In [ ]:
K=[2,3,4,5,6,7,8]
s_score=[silhouette_score(data_train_reduced, KMeans(n_clusters=i).fit(data_train_reduced).labels_) for i in K]
plt.plot(K,s_score, marker='o', color='g')
plt.xlabel('K')
plt.ylabel('Silhouette Score')

In [ ]:
plot_silhouette(K, data_train_reduced)

The highest silhouette score is on `n_cluster = 4`. However, if we see the visualization on the right side, it did not show a good clustering since we could see the green-colored cluster is mixed with the blue and yellow clusters. As well as other clusters, some clusters overlap with other clusters. 

The cluster that show a good visualization is `n_cluster = 2` and `n_cluster = 3`.  `n_cluster = 3` has higher silhouette score of 0.26 and it is higher than `n_cluster = 2` that has a silhouette score of 0.25. Therefore, for Kmeans Clustering will use `n_cluster = 3` to define the Kmeans Clustering model.

We will define Kmeans with `n_cluster = 3`

#### **6.1.2 Kmeans Clustering Model Definition**

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)

### **6.2 Agglomerative Clustering**

#### **6.2.1 Finding The Best Number of Cluster for Agglomerative Clustering**

To find the best cluster for Agglomerative Clustering, we will use dendogram and choose the longest lines that divide the clusters.

##### **Dendogram**

In [ ]:
plt.figure(figsize=(10, 7))  
plt.title("Dendrograms")  
dend = shc.dendrogram(shc.linkage(data_train_reduced, method='ward'))

Based on the dendogram above, we will use `n_cluster = 5` because there are 5 lines between 100 and 150 that divide the data into 5 groups.

#### **6.2.2 Agglomerative Clustering Model Definition**

In [ ]:
agglo = AgglomerativeClustering(n_clusters=5)

### **6.3 Gaussian Mixture Model (GMM)**

#### **6.3.1 Finding The Best Number of Component for GMM**

To find the best n_component (k) for GMM, we will calculate AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion). To determine the best `k` for GMM we will choose `k` with the lowest AIC.

In [ ]:
# Train GMM with Various Number of Clusters

gms_per_k = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(data_train_reduced)
             for k in range(1, 11)]

In [ ]:
# Get BIC and AIC Scores

bics = [model.bic(data_train_reduced) for model in gms_per_k]
aics = [model.aic(data_train_reduced) for model in gms_per_k]

for k in range(0, 10):
  print('Cluster : ', k+1, '\tBIC : ', bics[k], '\tAIC : ', aics[k])

In [ ]:
# Plot BIC Score and AIC Score

plt.figure(figsize=(10, 5))
plt.plot(range(1, 11), bics, "bo-", label="BIC")
plt.plot(range(1, 11), aics, "go--", label="AIC")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Information Criterion", fontsize=14)
plt.axis([1, 11, np.min(aics) - 50, np.max(aics) + 50])
plt.legend()
plt.show()

From the graph above, it appears that the `k` with the lowest AIC is **10**. However, in GMM we should also choose the best `covariance_type`, thus we will search for the best combination of values for both the `n_component` and `covariance_type` hyperparameter.

Since we can't use `GridSearchCV` for unsupervised learning, we will do 'for looping' to search for the best values of each hyperparameters.

In [ ]:
min_aic = np.infty

for k in range(1, 11):
    for covariance_type in ("full", "tied", "spherical", "diag"):
        aic = GaussianMixture(n_components=k, n_init=10,
                              covariance_type=covariance_type,
                              random_state=42).fit(data_train_reduced).aic(data_train_reduced)
        if aic < min_aic:
            min_aic = aic
            best_k = k
            best_covariance_type = covariance_type

print('best - n_components    : ', best_k)
print('best - covariance_type : ', best_covariance_type)

Just like the above plot, we get the best `n_component = 10`. For the `covariance_type`, the best value is `full`. According to Gaussian Mixture's documentation in scikit-learn.org, `full` means each component has its own general covariance matrix and it is the default `covariance_type`

#### **6.3.2 GMM Model Definition**

In [ ]:
gm = GaussianMixture(n_components=10, n_init=10, random_state=42, covariance_type='full')

## **7. Model Training**

### **7.1 Kmeans Clustering**

We will train the Kmeans Clustering with the data that has been reduced to 80% by using `PCA(n_components=0.8)`

In [ ]:
kmeans.fit(data_train_reduced)

Next, we will get the clusters in Kmeans Clustering.

In [ ]:
clusters_kmeans = kmeans.labels_

In [ ]:
kmeans.n_iter_

In this case, Kmeans Clustering only took 8 iterations to complete the modelling

### **7.2 Agglomerative Clustering**

We will train the Agglomerative Clustering with the data that has been reduced to 80% by using `PCA(n_components=0.8)`

In [ ]:
agglo.fit(data_train_reduced)

Next, we will get the clusters in Agglomerative Clustering.

In [ ]:
clusters_agglo = agglo.labels_

### **7.3 Gaussian Mixture Model (GMM)**

We will train the GMM with the data that has been reduced to 80% by using `PCA(n_components=0.8)`

In [ ]:
gm.fit(data_train_reduced)

Next, we will get the clusters in GMM.

In [ ]:
clusters_gm = gm.predict(data_train_reduced)

Did the algorithm already converged?

In [ ]:
gm.converged_

The model is converged, it means the model is already satisfied at a point.

In [ ]:
# Display Number of Step used to Reach the Convergence

gm.n_iter_

The GMM took 59 iterations to reach the convergence.

## **9. Model Evaluation**

First we will make reduce the dimension of `data_train_final` into 2 columns using `PCA(n_components=2)` and make a dataframe.

In [ ]:
# Dimensionality reduction to 2 columns
pca_2 = PCA(n_components=2)
pca_2.fit(data_train_final)
pca_2_tf = pca_2.transform(data_train_final)

# Make a dataframe
data_pca_df = pd.DataFrame(data = pca_2_tf, columns = ['PC 1', 'PC 2'])
data_pca_df.head()

### **9.1 Kmeans Clustering**

#### **9.1.1 Kmeans Clustering Visualization with PCA**

We will concatenate `data_pca_df` with `clusters_kmeans`.

In [ ]:
data_pca_kmeans = data_pca_df.copy()
data_pca_kmeans['clusters'] = clusters_kmeans
data_pca_kmeans.head()

Next, we will plot the clusters.

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

sns.scatterplot(
    x="PC 1", y="PC 2",
    hue="clusters",
    edgecolor='green',
    linestyle='--',
    data=data_pca_kmeans,
    palette='bright',
    ax=ax)

The plot above shows a good clustering. We can see that all three clusters are quite well separated, although we can still see some data points that crossed other clusters.

#### **9.1.2 EDA for Kmeans Clustering**

We will see the mean and median of each column based on the clusters.

In [ ]:
data_clustering_kmeans = data_train_ipt.copy()
data_clustering_kmeans['CLUSTERS'] = clusters_kmeans

In [ ]:
data_clustering_kmeans.groupby('CLUSTERS').agg(['mean','median'])

We can see in `BALANCE`, `PAYMENTS`, and `CREDIT_LIMIT ` the clusters have a fairly far and reasonable distance between the clusters. Based on these clusters, we can do customer segmentations to define marketing strategy.

In [ ]:
data_clustering_kmeans[['CLUSTERS']].value_counts()

Seeing from the `CREDIT_LIMIT`, there are 2227 customers in cluster 0 whose average credit limit is 5101.29, 4450 customers in cluster 1 whose average credit limit is 2937.98, and 2016 customers in cluster 2 whose average credit limit is 5781.41.

Customers that are in cluster 0 has low credit limit and low balance as well. It makes sense because if we see the pearson correlation coefficient in the EDA section, we can see that `CREDIT_LIMIT` has a strong correlation with `BALANCE`.

In [ ]:
data_clustering_kmeans[['PURCHASES_FREQUENCY', 'CLUSTERS']].groupby('CLUSTERS').mean()

### **9.2 Agglomerative Clustering**

#### **9.2.1 Agglomerative Clustering Visualization with PCA**

We will concatenate `data_pca_df` with `clusters_agglo`.

In [ ]:
data_pca_agglo = data_pca_df.copy()
data_pca_agglo['clusters'] = clusters_agglo
data_pca_agglo.head()

Next, we will plot the clusters.

In [ ]:
fig1, ax1 = plt.subplots(figsize=(15,10))

sns.scatterplot(
    x="PC 1", y="PC 2",
    hue="clusters",
    edgecolor='green',
    linestyle='--',
    data=data_pca_agglo,
    palette='bright',
    ax=ax1)

The clustering result above is not really good. As we can see above cluster 3 are spread all over the data points, same goes for cluster 4. We cannot really decide which data lies in which clusters because it is too overlapping and confusing.

#### **9.1.2 EDA for Agglomerative Clustering**

In [ ]:
data_clustering_agglo = data_train_ipt.copy()
data_clustering_agglo['CLUSTERS'] = clusters_agglo

In [ ]:
data_clustering_agglo.groupby('CLUSTERS').agg(['mean','median'])

In [ ]:
data_clustering_agglo[['CLUSTERS']].value_counts()

We have 5 clusters generated by the Agglomerative Clustering. We have most customers in cluster 0, which shows a moderate `BALANCE` and `CREDIT_LIMIT`. And we have least customers in cluster 3, which shows low `BALANCE` and `CREDIT_LIMIT`. 

This clustering is not really reliable because we can see from the plot above the clusters are not well separated and very overlapping with each other.

### **9.3 Gaussian Mixture Model (GMM)**

#### **9.2.1 GMM Visualization with PCA**

We will concatenate `data_pca_df` with `clusters_gm`.

In [ ]:
data_pca_gm = data_pca_df.copy()
data_pca_gm['clusters'] = clusters_gm
data_pca_gm.head()

Next, we will plot the clusters.

In [ ]:
fig2, ax2 = plt.subplots(figsize=(15,10))

sns.scatterplot(
    x="PC 1", y="PC 2",
    hue="clusters",
    edgecolor='green',
    linestyle='--',
    data=data_pca_gm,
    palette='bright',
    ax=ax2)

The visualization is very confusing. We really cannot see the boundaries of each cluster. The clusters are very overlapping and confusing.

#### **9.1.2 EDA for GMM**

We will see the mean and median of each column based on the clusters.

In [ ]:
data_clustering_gm = data_train_ipt.copy()
data_clustering_gm['CLUSTERS'] = clusters_gm

In [ ]:
data_clustering_gm.groupby('CLUSTERS').agg(['mean','median'])

In [ ]:
data_clustering_gm[['CLUSTERS']].value_counts()

We have 10 clusters generated by GMM. It is quite too many for doing segmentation with such number of clusters. As we can see in the table above, all mean and median in `BALANCE` and `CREDIT_LIMIT` did not differ that much, so we cannot really see the difference of each cluster.

## **9. Model Inference**

For model inference, we will use Kmeans Clustering, because it showed the best clustering.

#### **Handling Outliers**

In [ ]:
data_inf.shape

In [ ]:
for i in num_data:
    detect_otl(data_inf, i)

In [ ]:
inf_otl_cap = ['MINIMUM_PAYMENTS', 'PAYMENTS', 'PURCHASES_TRX', 'CASH_ADVANCE_TRX', 'CASH_ADVANCE_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY', 
               'CASH_ADVANCE', 'INSTALLMENTS_PURCHASES', 'PURCHASES', 'BALANCE', 'PRC_FULL_PAYMENT', 'CREDIT_LIMIT' ]

In [ ]:
winsorizer = Winsorizer(capping_method='iqr',
                        tail='both',
                        fold=1.5,
                        variables=inf_otl_cap,
                        missing_values='ignore')

data_inf_capped = winsorizer.fit_transform(data_inf)

#### **Handling Missing Values**

In [ ]:
data_inf_capped.isnull().sum()

#### **Feature Scaling**

We will exclude `TENURE` from feature scaling because it is a categorical data.

In [ ]:
inf_for_scaling = data_inf_capped.copy()
inf_for_scaling.drop('TENURE', axis=1,inplace=True)

In [ ]:
standard_scaler = StandardScaler()

data_inf_scaled = standard_scaler.fit_transform(inf_for_scaling)

In [ ]:
data_inf_scaled_df = pd.DataFrame(data_inf_scaled, columns=inf_for_scaling.columns)
data_inf_scaled_df['TENURE'] = data_inf_capped['TENURE']

In [ ]:
data_inf_final = data_inf_scaled_df.to_numpy()
data_inf_final.shape

#### **PCA for Model Inference**

Because our kmeans model is already fit with 6 features, so for PCA in the inference model we directly use `PCA(n_components = 6)`

In [ ]:
pca_inf = PCA(n_components=6).fit_transform(data_inf_final)

In [ ]:
pca_inf.shape

In [ ]:
inf_cluster = kmeans.predict(pca_inf)

In [ ]:
data_clustering_inf = data_inf_capped.copy()
data_clustering_inf['CLUSTERS'] = inf_cluster

In [ ]:
data_clustering_inf.groupby('CLUSTERS').agg(['mean','median'])

Same with the train model, the clustering in Model Inference shows a good result. We can see the difference between each cluster and do the segmentation well. However, we can't really visualize this model inference because the size of the data is too small, we won't be able to see the clusters clearly.

## **10. Conclusion**

In this case, Kmeans Clustering showed better clustering result than Agglomerative Clustering and Gaussian Mixture Model. The clusters have clear boundaries, we can really see the difference of each cluster. 

For Agglomerative Clustering, in this case it did not perform well. The clustering result was confusing and it is quite difficult to see the boundaries of each cluster. Same goes for Gaussian Mixture Model, we could not really see the boundaries of each cluster and it is so hard to understand the clusters.

For further busines purposes, it is recommended to use Kmeans Clustering to do a customer segmentation. We already have 3 clusters, and based on the `CREDIT_LIMIT` we can segment the customers with low, moderate, and high credit limits. Based on this segmentation, we can start to plan our marketing strategy. We can also segment the customers based on the other behavioral variables, such as `BALANCE`, `PURCHASE`, etc.